In [1]:
import json

In [2]:
filepath = "/Users/eileentoomer/Code/runningData/src/runningData/data/night10k2019.json"
fileToSave = "/Users/eileentoomer/Code/runningData/src/runningData/data/night10k2019_transformed.json"

with open(filepath) as f:
    data = json.load(f)

#### Competitors:

In [3]:
competitors = data['competitors']

In [4]:
athletes = {}

for c in competitors:
    athletes[int(c['sortBib'])] = {
        'name': "{f} {s}".format(f=c['firstName'], s=c['lastName']),
        'teamId': c['teamId']
    }
    

### Results

In [5]:
def parseRaceTime(s):
    t = [float(i) for i in s.split(':')]
    if len(t) == 1:
        return round(t[0], 2)
    else:
        return round(t[0]*60 + t[1], 2)

In [6]:
def getAthleteRaceSplits(d):
    splits = []
    t0 = 0
    for s in d['splits']:
        t = parseRaceTime(s['elapsedTime'])
        splits.append({
            'lap': int(s['distance'] / 200),
            'distance': s['distance'],
            'seconds': t,
            'elapsedTime': s['elapsedTime'],
            'lapTime': t - t0
        })
        
        t0 = t
        
    if splits[len(splits) - 1]['lap'] == 49:
        t = parseRaceTime(d['performance'])
        splits.append({
            'lap': 50,
            'distance': 10000,
            'seconds': t,
            'elapsedTime': d['performance'],
            'lapTime': t - t0
        })
        
    return splits   

In [7]:
def getRaceSplits(data): 
    race_results = []
    
    for d in data:
        if (d['performance'] not in ['DNF', 'DNS']):
            if (len(d['splits']) > 30):
                race_results.append({
                    'bib': int(d['bib']),
                    'athlete': athletes[int(d['bib'])]['name'],
                    'club': athletes[int(d['bib'])]['teamId'],
                    'performance': d['performance'],
                    'seconds': 'DNF' if d['performance'] in ['DNF', 'DNS'] else parseRaceTime(d['performance']),
                    'position': d['place'],
                    'splits': getAthleteRaceSplits(d)
                })
        
    return race_results

In [8]:
def addLapPositions(race):

    elapsedTimes = {}
    for a in race:
        for l in a['splits']:
            if l['lap'] in elapsedTimes.keys():
                elapsedTimes[l['lap']].append(l['seconds'])
            else:
                elapsedTimes[l['lap']] = [l['seconds']]
                
    for k, v in elapsedTimes.items():
        elapsedTimes[k].sort()

    #for a in race:
    for a in race:
        for l in a['splits']:
            currentPosition = elapsedTimes[l['lap']].index(l['seconds']) + 1
            l.update({'currentPosition': currentPosition})
            
    return race
        

In [9]:
race = getRaceSplits(data['events'][8]['units'][0]['results'])
race = addLapPositions(race)

In [10]:
from operator import itemgetter
newlist = sorted(race, key=itemgetter('position'))

In [11]:
allRaceResults = {}
r = 0
for d in data['events'][2:]:
    if d['eventCode'] == '10000':
    
        race = getRaceSplits(d['units'][0]['results'])
        race = addLapPositions(race)

        allRaceResults[r] = {
                'race': d['name'],
                'splits': sorted(race, key=itemgetter('position'))
        }
        r += 1
        

In [23]:
with open(fileToSave, 'w') as fp:
    json.dump(allRaceResults, fp)

In [21]:
for a in allRaceResults[3]['splits']:
    print(a['athlete'] + f": {len(a['splits'])}" )

Mohamed Mohamed: 50
Stefan Iulius Gavril: 50
Omar Ahmed: 50
Joshua Grace: 50
Callan Moody: 50
Nigel Martin: 50
Ollie Lockley: 50
Carl Avery: 50
Michael Christoforou: 50
Matt Clowes: 50
Ben Cole: 50
Austen Dalquist: 50
Tim Bolink: 50
Nick Earl: 50
Phillip Crout: 50
Norman Shreeve: 50
Nicholas Torry: 50
Jonathan Cornish: 50
Corey De'Ath: 50
Ryan Thomson: 50
Harry Allen: 50


In [14]:
import numpy as np

def formatTime(t):
    m = int(np.floor(t / 60))
    s = round(t % 60, 2)
    
    if s < 10:
        return f"{m}:0{s}"
    else:
        return f"{m}:{s}"

In [15]:
athlete_positions = []
athlete_names = []

for a in allRaceResults[1]['splits']:
    p0 = {'currentPosition': 0, 'seconds': 0}
    name_list = a['athlete'].split(" ")
    n = "".join([a[0] for a in name_list])
    if n in athlete_names:
        n = "".join([name_list[0][0], name_list[1][0:2]])
    athlete_names.append(n)
    for i in range(9):
        lap = (i)*5 + 4
        p1 = a['splits'][lap]
        t = formatTime(p1['seconds'] - p0['seconds'])
        
        athlete_positions.append([f"{n} {(i)}km {n}", f"{n} {(i+1)}km", 1, f"{a['athlete']} ({t})", lap, p0['currentPosition'], p1['currentPosition']])
        
        p0 = p1

athlete_positions.sort(key=lambda k: (k[4], k[6]))
[a[:4] for a in athlete_positions]
        

[['ET 0km ET', 'ET 1km', 1, 'Eoghan Totten (2:59.38)'],
 ['JW 0km JW', 'JW 1km', 1, 'Joe Wilkinson (2:59.51)'],
 ['YD 0km YD', 'YD 1km', 1, 'Yannik Duppich (2:59.58)'],
 ['JF 0km JF', 'JF 1km', 1, 'Jay Ferns (2:59.77)'],
 ['AÅ 0km AÅ', 'AÅ 1km', 1, 'Andreas Åhwall (2:59.79)'],
 ['DM 0km DM', 'DM 1km', 1, 'Daniel Mulryan (3:00.1)'],
 ['PW 0km PW', 'PW 1km', 1, 'Paul Whittaker (3:00.27)'],
 ['RS 0km RS', 'RS 1km', 1, 'Ross Skelton (3:00.77)'],
 ['JLe 0km JLe', 'JLe 1km', 1, 'Jack Leitch (3:01.28)'],
 ['JL 0km JL', 'JL 1km', 1, 'Josh Lunn (3:01.41)'],
 ['GJ 0km GJ', 'GJ 1km', 1, 'Geart Jorritsma (3:01.43)'],
 ['JD 0km JD', 'JD 1km', 1, 'James Douglas (3:01.74)'],
 ['SF 0km SF', 'SF 1km', 1, 'Sean Fontana (3:01.93)'],
 ['RB 0km RB', 'RB 1km', 1, 'Ross Braden (3:02.26)'],
 ['WR 0km WR', 'WR 1km', 1, 'William Ryle-Hodges (3:02.89)'],
 ['PR 0km PR', 'PR 1km', 1, 'Philip Robertson (3:03.17)'],
 ['TD 0km TD', 'TD 1km', 1, 'Thomas Drabble (3:03.29)'],
 ['BJ 0km BJ', 'BJ 1km', 1, 'Ben Johnson (3: